# Code Wasteground

This is a notebook containing test code, nothing to see here.

# Setup

In [0]:
SEQ_LEN = 100  # maximum sequence length we want for training
EPOCHS = 12  # number of times we iterate over the full dataset during training
RNN = 'GRU'  # whether we use LSTM or GRU RNN units
UNITS = 2048  # how many units we use
BATCH_SIZE = 64  # no. sequences of SEQ_LEN we train on before updating weights
EMBED_DIM = 256  # vector dimension of word vector embeddings
PRINT = 1000  # how many characters we print during text generation








# 1. Importing Modules and Data

In [0]:
import requests
import re
import numpy as np

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import os

In [0]:
# import Meditations by Marcus Aurelius
response = requests.get('http://classics.mit.edu/Antoninus/meditations.mb.txt')
data = response.text
del response

In [4]:
print(data[:500])

Provided by The Internet Classics Archive.
See bottom for copyright. Available online at
    http://classics.mit.edu//Antoninus/meditations.html

The Meditations
By Marcus Aurelius


Translated by George Long

----------------------------------------------------------------------

BOOK ONE

From my grandfather Verus I learned good morals and the government
of my temper. 

From the reputation and remembrance of my father, modesty and a manly
character. 

From my mother, piety and beneficence, and


# 2. Pre-Processing Data
## 2.1 Cleaning
### 2.1.1 Filter Out
There are several items that we must filter out from Meditations, these are:
<ul>
    <li>everything before the first "---------" line (or everything before and including "Translated by George Long")</li>
    <li>any line that says "BOOK ..."</li>
    <li>any line that is "---------"</li>
    <li>"THE END" and everything after this</li>
</ul>

In [5]:
# remove everything before and including "Translated by George Long"
data = data.split('Translated by George Long')[1]
print(data[:500])



----------------------------------------------------------------------

BOOK ONE

From my grandfather Verus I learned good morals and the government
of my temper. 

From the reputation and remembrance of my father, modesty and a manly
character. 

From my mother, piety and beneficence, and abstinence, not only from
evil deeds, but even from evil thoughts; and further, simplicity in
my way of living, far removed from the habits of the rich.

From my great-grandfather, not to have frequented pub


In [6]:
# remove "----" lines, as "-" is not a useful character we will remove it completely
data = data.replace('-', '')
print(data[:500])





BOOK ONE

From my grandfather Verus I learned good morals and the government
of my temper. 

From the reputation and remembrance of my father, modesty and a manly
character. 

From my mother, piety and beneficence, and abstinence, not only from
evil deeds, but even from evil thoughts; and further, simplicity in
my way of living, far removed from the habits of the rich.

From my greatgrandfather, not to have frequented public schools,
and to have had good teachers at home, and to know that on


In [7]:
# remove "BOOK ..." lines, for this we use regular expressions
data = re.sub('BOOK [A-Z]+\n', '', data)
print(data[:500] + "\n\n...\n\n" + data[-500:])






From my grandfather Verus I learned good morals and the government
of my temper. 

From the reputation and remembrance of my father, modesty and a manly
character. 

From my mother, piety and beneficence, and abstinence, not only from
evil deeds, but even from evil thoughts; and further, simplicity in
my way of living, far removed from the habits of the rich.

From my greatgrandfather, not to have frequented public schools,
and to have had good teachers at home, and to know that on such thi

...

show

THE END



Copyright statement:
The Internet Classics Archive by Daniel C. Stevenson, Web Atomics.
World Wide Web presentation is copyright (C) 19942000, Daniel
C. Stevenson, Web Atomics.
All rights reserved under international and panAmerican copyright
conventions, including the right of reproduction in whole or in part
in any form. Direct permission requests to classics@classics.mit.edu.
Translation of "The Deeds of the Divine Augustus" by Augustus is
copyright (C) Thomas Bushnell

In [8]:
# remove "THE END" and all that follows it
data = data.split("THE END")[0]
print(data[:500] + "\n\n...\n\n" + data[-500:])






From my grandfather Verus I learned good morals and the government
of my temper. 

From the reputation and remembrance of my father, modesty and a manly
character. 

From my mother, piety and beneficence, and abstinence, not only from
evil deeds, but even from evil thoughts; and further, simplicity in
my way of living, far removed from the habits of the rich.

From my greatgrandfather, not to have frequented public schools,
and to have had good teachers at home, and to know that on such thi

...

sh and a tale, or not even a tale.
And let there be present to thy mind also everything of this sort,
how Fabius Catullinus lived in the country, and Lucius Lupus in his
gardens, and Stertinius at Baiae, and Tiberius at Capreae and Velius
Rufus (or Rufus at Velia); and in fine think of the eager pursuit
of anything conjoined with pride; and how worthless everything is
after which men violently strain; and how much more philosophical
it is for a man in the opportunities presented to him to

### 2.1.2 Pull Character-Level Vocab

We do this before splitting the data for character level

In [9]:
vocab_char = sorted(set(data))  # character level vocab
print(f'{len(vocab_char)} unique characters found')

66 unique characters found


In [10]:
print(vocab_char)

['\n', ' ', '!', '"', "'", '(', ')', ',', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


### 2.1.3 Split by Paragraph

In [11]:
# splitting by newline characters
data = data.split('\n\n')
print(data[:10])

['', '', '\nFrom my grandfather Verus I learned good morals and the government\nof my temper. ', 'From the reputation and remembrance of my father, modesty and a manly\ncharacter. ', 'From my mother, piety and beneficence, and abstinence, not only from\nevil deeds, but even from evil thoughts; and further, simplicity in\nmy way of living, far removed from the habits of the rich.', 'From my greatgrandfather, not to have frequented public schools,\nand to have had good teachers at home, and to know that on such things\na man should spend liberally. ', "From my governor, to be neither of the green nor of the blue party\nat the games in the Circus, nor a partizan either of the Parmularius\nor the Scutarius at the gladiators' fights; from him too I learned\nendurance of labour, and to want little, and to work with my own hands,\nand not to meddle with other people's affairs, and not to be ready\nto listen to slander. ", 'From Diognetus, not to busy myself about trifling things, and not\nto 

In [12]:
# remove empty samples
empty = lambda x: x.replace('\s+', '') != ''
data = list(filter(empty, data))
print(data[:10])

['\nFrom my grandfather Verus I learned good morals and the government\nof my temper. ', 'From the reputation and remembrance of my father, modesty and a manly\ncharacter. ', 'From my mother, piety and beneficence, and abstinence, not only from\nevil deeds, but even from evil thoughts; and further, simplicity in\nmy way of living, far removed from the habits of the rich.', 'From my greatgrandfather, not to have frequented public schools,\nand to have had good teachers at home, and to know that on such things\na man should spend liberally. ', "From my governor, to be neither of the green nor of the blue party\nat the games in the Circus, nor a partizan either of the Parmularius\nor the Scutarius at the gladiators' fights; from him too I learned\nendurance of labour, and to want little, and to work with my own hands,\nand not to meddle with other people's affairs, and not to be ready\nto listen to slander. ", 'From Diognetus, not to busy myself about trifling things, and not\nto give cre

In [0]:
# remove final '\n' characters
data = list(map(lambda x: x.replace('\n', ' '), data))

In [14]:
data[:20]

[' From my grandfather Verus I learned good morals and the government of my temper. ',
 'From the reputation and remembrance of my father, modesty and a manly character. ',
 'From my mother, piety and beneficence, and abstinence, not only from evil deeds, but even from evil thoughts; and further, simplicity in my way of living, far removed from the habits of the rich.',
 'From my greatgrandfather, not to have frequented public schools, and to have had good teachers at home, and to know that on such things a man should spend liberally. ',
 "From my governor, to be neither of the green nor of the blue party at the games in the Circus, nor a partizan either of the Parmularius or the Scutarius at the gladiators' fights; from him too I learned endurance of labour, and to want little, and to work with my own hands, and not to meddle with other people's affairs, and not to be ready to listen to slander. ",
 'From Diognetus, not to busy myself about trifling things, and not to give credit to w

In [15]:
print(f"We have {len(data)} stoic lessons from Marcus Aurelius")

We have 507 stoic lessons from Marcus Aurelius


In [0]:
# now join back together in full text
data = '\n'.join(map(lambda x: x.strip(), data))  # we also use map to strip each paragraph

### 2.1.4 Pull Word-Level Vocab

In [17]:
vocab_word = sorted(set(data.split(' ')))  # word level vocab
print(f'{len(vocab_word)} unique words found')

# !!! TODO: add punctuation as it's own word (if we go with word-level)

6454 unique words found


## 2.2 Format Preparation

### 2.2.1 Mapping Tables

In [0]:
char2idx = {u:i for i, u in enumerate(vocab_char)}
idx2char = np.array(vocab_char)

In [0]:
example_idx = np.array([char2idx[c] for c in "Everything we see is a perspective, not the truth."])

In [20]:
print(example_idx)

[16 61 44 57 64 59 47 48 53 46  1 62 44  1 58 44 44  1 48 58  1 40  1 55
 44 57 58 55 44 42 59 48 61 44  7  1 53 54 59  1 59 47 44  1 59 57 60 59
 47  8]


In [0]:
data_idx = np.array([char2idx[c] for c in data])

In [22]:
len(data[0])

1

### 2.2.2 Slicing Data
For the training data, we divide the text into example sequences. Each input sequence is of length <b>seq_length</b>. Each corresponding target sequence will consist of the input sequence shifted one character to the right.

In [23]:
len(data)

241199

In [0]:
# max length sentence we want
seq_length = SEQ_LEN
examples_per_epoch = len(data) // (seq_length+1)

# create training data
char_dataset = tf.data.Dataset.from_tensor_slices(data_idx)

In [0]:
# batch method allows conversion of individual characters to sequences of a desired size
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [0]:
# for each sequence, we duplicate and shift it to form the input and target text
def split_input_output(chunk):
    input_data = chunk[:-1]
    target_data = chunk[1:]
    return input_data, target_data

dataset = sequences.map(split_input_output)

### 2.2.3 Training Batches

Before feeding data to the model, must shuffle the data and pack into batches

In [27]:
# buffer size to shuffle the dataset
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

# 3 Build Model

In [0]:
def build_model(vocab_size, embed_dim, rnn_units, batch_size):
    if RNN == 'GRU':
        model = tf.keras.Sequential([
            tf.keras.layers.Embedding(vocab_size, embed_dim,
                                    batch_input_shape=[batch_size, None]),
            tf.keras.layers.GRU(rnn_units, return_sequences=True,
                                stateful=True,
                                recurrent_initializer='glorot_uniform'),
            tf.keras.layers.Dense(vocab_size)
        ])
    elif RNN == 'LSTM':
        model = tf.keras.Sequential([
            tf.keras.layers.Embedding(vocab_size, embed_dim,
                                    batch_input_shape=[batch_size, None]),
            tf.keras.layers.LSTM(rnn_units, return_sequences=True,
                                 stateful=True,
                                 recurrent_initializer='glorot_uniform'),
            tf.keras.layers.Dense(vocab_size)
        ])
    return model

In [29]:
model = build_model(
    vocab_size=len(vocab_char),
    embed_dim=EMBED_DIM,
    rnn_units=UNITS,
    batch_size=BATCH_SIZE)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16896     
_________________________________________________________________
gru (GRU)                    (64, None, 2048)          14168064  
_________________________________________________________________
dense (Dense)                (64, None, 66)            135234    
Total params: 14,320,194
Trainable params: 14,320,194
Non-trainable params: 0
_________________________________________________________________


## Note:

For some reason, the same model built another way causes an error when generating text. I have asked why on the following Stackoverflow page:


The alternative code causing the error is:



```
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embed_dim,
                                        batch_input_shape=[BATCH_SIZE, None]))
if RNN == 'GRU':
    model.add(tf.keras.layers.GRU(UNITS, return_sequences=True,
                                  stateful=True,
                                  recurrent_initializer='glorot_uniform'))
else:
    model.add(tf.keras.layers.LSTM(UNITS, return_sequences=True,
                                   stateful=True,
                                   recurrent_initializer='glorot_uniform'))
model.add(tf.keras.layers.Dense(vocab_size))
```



# 4. Training

In [0]:
# define the loss function
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [0]:
# compile the model
model.compile(optimizer='adam', loss=loss)

In [0]:
# configure checkpoints
checkpoint_dir = './training_checkpoints'
# name of files
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [0]:
# execute training
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/12
37/37 [==============================] - 697s 19s/step - loss: 3.8087
Epoch 2/12
37/37 [==============================] - 694s 19s/step - loss: 2.4964
Epoch 3/12
37/37 [==============================] - 712s 19s/step - loss: 2.3025
Epoch 4/12
 7/37 [====>.........................] - ETA: 8:22 - loss: 2.2263

# 5. Generate Text

## 5.1 Restore Final Checkpoint
Because of the way the RNN state is passed from timestep to timestep the model only accepts a fixed batch size once built.

To run the model with another batch_size we rebuild to model and restore weights from the checkpoint

<b>maybe this can be done in Tensorflow.js?</b>

In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

In [0]:
model = build_model(len(vocab_char), EMBED_DIM, UNITS, 1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

In [0]:
# defining text generation function
def generate_text(model, start_string, length=1000, temp=1.0):
    # low temp = more predictable text, high temp = more surprising text

    # converting start string to numbers (vectorisation)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # initialise empty string to store results
    text = []

    # batch size is 1
    model.reset_states()
    for i in range(length):
        predictions = model(input_eval)
        # remove batch dimension
        predictions = tf.squeeze(predictions, 0)

        # use categorical distribution to predict character returned by model
        predictions = predictions / temp
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # we pass the predicted character as the next input to the model
        # along with previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        # append predicted text
        text.append(idx2char[predicted_id])

    return (start_string + ''.join(text))

In [0]:
print(generate_text(model, start_string=u'From ', length=PRINT))

In [0]:
history

# 12 iterations
## GRU

<b>From</b> any placest is so.

Thou art dods thee from the deadgh of impuety by habour of mecrating, and oter coused by doing with gaves indifected, if thou hass serdect to him at a think. The way only not ando, nothing is compousted the that happoness; and de to be exturn and I he another suchivisicis inforaition with its Iw all vellected with does at all things, how sefull: and kill not the beact?

That this a soun of the saie past, but lothing else is gorally that they relamas; but thy chatical and injiencque its shall goderatis, or one oppens; and how the raying phatiet of thy negerce, and a stead. And us our bears prriaced from however suagled with the places with many by other by social interest findly op not deity it is of a way a man it has subce into says of a shall call the thing in thems experacled which good feruble them, but Alsose and blamon, and welse hassides, fruedome power life, them wouldsh tranfining procurnation.

There would harm he whole could a portur and a reculsm for this i

# 24 iterations
## GRU
### 1024 Units

<b>From</b> the intelligent collision, and to his consistlation or unfustiles respect to the rapidity, caln: thou, wilt last to live prevent through a good man, us afor on to that eit when has myser his present quuriely? For own nects fortus I have completed, to discover that thou mayest also does no harm to this.

Not turn to thee consist of nature is a thing, for it truly see that thou thyself still firl into them. Altogether to come at thing is a good forte: and if thou to me the cause; but in the sturce that my cherich are honextife, it would thou not seek flattering of thy nature that all the things which are simple, the strangerswat man's intelligence and yet died every men keep myself, and so affectionate, and as then cannothe other social sopical or God.

The reason resolutes fer investiguted without sufficient to everything more, boccolds to everything which is oblivion, and to evil to my excare imposed to me, thou are the appettable, thou wilt recorce so let not too. In the simple which ha

## LSTM

<b>From</b> understand whom they dissolution died. Further, neither is a rocernal counterest, come for thee are formed by nature; for they are when it both in thy power. Well, sump things with been, he should sespord, but in so is shown; whe must discovering thy benevolent des with see; but that it some within without tre with that with which thou hast seen Another in my opinions. And thou hadst separated from life.

Tourght of ndire, so many after asidged his portion, or down and of that which caul and ready eythy of one according to existence out of what is not caul say or Paway of or altog, and that it will be either affairates, and hon that thou hast been a knowledoe, and in can, Be in a handreit for something else.

Phar has been constituted, firellesturiou was its own act chance of him, and remedness of which a man beings what happens and in sumpersh, throesonu, will be obssiedly when thou arother, shall care after different things; and he choose there of whom universal, death, or is intellige

# 48 iterations
## GRU
### 1024 Units

<b>From</b> of those who live with us and speak; and that we ought not, like children.

The Lace after the preshn and all to the state, do not to be such as the whole of thy life. For as soon as I have freedom of which everything harmbeing and deprived of the power of bistence? And if they get as doing at the same time having a reference to the intelficable and profitable! And yet have no rouses and us unter out of the Ephesians there was this precept, thou mightest remember that thy attempt was right to vexattroperate for these purpose. And this farout inquiry, moves strain those who have d soul, andore erect, not be deprived of the soul of man, and to the soul of God.

All things are the condidion of the universe to be none by any exalted. For one evolence is it with reference to the thouard, thou wilt see what I say, if thou only desire to be a good man in earthy turns be the imagination. Stop the pulling of the strings. Confine thyself to the things which are presented to it, st parts, whicerdon

### 2048 Units

...

# 64 Iterations

## 1024 Units
### SEQ_LEN = 100

From many together with more quiet or more freedom from trouble does no harm to the elements themselves in the soul, if it be consistent with social interests, to the Universe will be example and blate with my own hands, and not to be seen by the gods neither dissatisfied with any solid body which stands, and be to consider that the men of after time which are in out to the earse (reason; and the activities of things are in consequence of change.

Time is useful to thee and words it, and do not for thishout turning back: but if thou canst shom which a vert excuse d good quirity are on om roo.
kin, and to look for the fig in winter is, which has changed, and that it without deathred any at back; and whet have designed.

Does another do me wrong? Let him look to this only in all that he does, whether he is doing what is just, and so secure the saying of Hy nature, and everything else is of the man as if wo far as soon as thou findest many other differences at the games in which thou hast abvert

### SEQ_LEN = 300

From I mout dyems; but let this prescribed this kind andorgance is a polity which is not life, retarn to thyself, motever, they to care atail for the poor bless society. Now mutation of epicials accuptom that no man would not have the country of the food free from all emptying sufficient confly than to be hindered. Thou wilt see how a fightence just see is an emply things both flate and life, if immediately passed the opinion? And accordingly men soone then being hadden's activity, and towards mutual compair in my opinion, thou also when the whill in his deserts, a right the desires of are, it being contentment with him who have right as the mottern thres himself to other things.

To power is more that his life indoed instruption at all to those who brongs the bad man do not being reverence to oureche is an object. So far as he should pragree some other naburity have been mistered? We hust thee that these persons so die, it last bo it. And he has a mea human being, having with mid and felici


# Hypothesis
Increasing SEQ_LEN increases the continuity of language, but requires more iterations in order to understand smaller sections of language as well as decreased SEQ_LEN values.

> Try SEQ_LEN=300 with EPOCHS=128

# 128 Iterations
### SEQ_LEN = 300

From Sheersally so faultain by it.

Let him look to it. But I will be mild and benevolent towards all things which man for those hourself, I shall bear in mind that he is compelled to do so.

Remember that as it is a shame to be surprised if the figtree produces figs, so it is to be surprised, if a man has a fever, or if it were the last, laying aside all carelessness and passionate aversion from the commands of reason, well then, if it is useful to thee as a rational being, keep to it; but if it is only useful to the universal nature. And consider that as the heasons of thy life bring, such also is dissolution. This, then, is consistency wat full in life bousest for a hundred years or threaf and the constituted for one cooperation. And the perception dull, and the composition of the whole body subject to putrefaction, and the soul a whirl, and fortune hard to divine, and fame aw being valued by them; nor in its nothing which it does not allow the fiety and free and modesty and with men, they

# 254 Iterations

## SEQ_LEN = 300

From my grandfather Verus I learned good morals and the government of my temper.

From the reputation and remembrance of my father, modesty and a manly character.

From my mother, piety and beneficence, and abstinence, not only from evil deeds, but even from evill things will perish into honour, which they seemed to desire, without putting them off with hope of my doing it some time after, because they were then still young; that I knew Apollonius, Rustice, and to hish they are more such effort as this. But thou must exent to be accomplished on the illness of all things, and he is Law, and assigns to every man what is fit.

He who has a vehement desire for posthumous fame does not consider that every one of those who remember him will himself also die very soon; then again also they who have succeeded them, until the whole remembrance shall have been extinguished as it is transmitted through men who fears or is grieved opportunity offers, thou fend towards them, for by nature they are friends. And the gods too aid them in all ways, by dreams, by signs, towards the attainment of those things on which they set a value.

The periodic movements of the universe are the same, up and down from age to age. And either the universal intelligence puts itself, I man directs its own movement are governed by no rational principle. If this is remembered it will make thee more tranquil in many things are suitable to us, arm to the deity, so far as I am A city as in thy power? Take away these opinions then, and resolve to despise them, and to do good to men, and to practise tolerance to an act; and that which is an obstacle on the road helps us on this road.

Reverence that which is diseased.

Shortlived are both the praiser and the praised, and the rememberer and the dead word of what thou hast known, one after another. One man after burying another has been laid out dead, and another buries him: and all that is allotted, as coming from thence, wherever it is, from whence he himself came; and, finally, waiting for death with a cheerful men or irrational? Souls of rational men. Sound or unsound? Sound. Why then do you not seek for them? Because thou hast seen Xany indeed have not been remembered even for a short time, and others have become the heroes of fables, and again others have disappeared even from fables. Remember this then, that this little compound, thyself, must either be dissolved, or thy duty to comf to thyself: For thou art a part. But that his own nature. And it is in his power to do according to his own nature. And it is in his power everywhere. Now, it is not given to a cylinder towards the venything of this knether thing to the peculiarly in old, are now in a manner antiquated, Camillus, Caeso, Volesus, Leonnatus, and a little after also Shing which happens is a separation of ourselves from nature, in some part of which the natures of all other things are contained. In the next place, the soul does violence to itself when it turns away from any man, or eve

# Note
I imagine this is overfitted, it is essentially copy and paste from the original text unfortunately, so maybe shorter SEQ_LEN and less iterations. Loss reached ~0.05.

# 50 SEQ_LEN tests
## 64 Iters
### LSTM

...

### GRU

From my grandfather Verus I leach e the ligh of the rational animal is common also to irrational animal. And this faculty produced, to be pulled by the branch must of necessity be. For who is allotted to him, and wild be said, do not regull thee no not then st art than the dancer the death of a single man, but of it done another says, cividect, and he had the faculty both in the whole body. But all of these phin it was, a Ruman bodil then is the ruling part: consider thus: Thou art a how small a part of such a whole, It is nature whichos shall I not desire to be released? Another thus: However I am goid of time can be the cause; and let there be justice in the things which happen to thee with his nails, and by dashing against thee as much as pain, and let neither another man's activity to be his own activity. But what the way which leads to the enembrance will be immortal, what then is this to thee to be released from this? Why dost thou in all cases call that a be overpowered with any affectation: use flatter one another then always abticce again makes no difference whether a man shall hinder thee from being me touched either berence which are of one who knows not however what is according to the perfect principles, and above that power which adain will stone daemon within which all things disappear, if thou doest every act of thy life as in order that these affect in thee and justice and the pancratium. In all things, then, expect the inferior things for a hundred years or ndwring else, indeparated from another may happen to them, and see how many after great efor not only unnecessary acts, of this is always renewingithem nor demanated profitable be bad.

No joinonopo which he is a str, and friendly and benevolence and freedom frong death, and disputed more skilfully with the source of all things, from which allo should propose in the counthim, or has been placed by a commands, and fored valued becemety also, uncontinually. In the ner thou hast forgotten this, that everything which is common to them all means beautiful in itself, and terminates in ill no less plainly.

Constantly consider how all things are the same, familiar in none of the things which are not within thy power all things, and contains and embraces all things, and contains its of substance and of time, and of motion and of the mind so that thou mayest make it just as if the atof all for a short time, and then life will be at an inquire about that which ought to be called about them alman and a man to do when thou hast done a man a state within why art thou disturbed? What then dies who is in fact it is compounded, and to be done a separation. And something else...

Everything which happens and is assigned to each man is carried alanter in thy collection.

For what must a man do who thee back free from all did anycontimately arr principles of the seasons each thing its value. Observe then as thou didst come, for I be no reason. And I am in a manner intimately related to the parts which are of the same kind as he see not even against their will, and to employ event from the work and ghought according to nature. If they cannot endure him, labour igs own.

Consider that everybod

Nothing is with respect to all things act with one another the things which are d way what is not in conformity to a man a thing of brigence wilt thou mean of my children if the gods negligence is carefully exercise of art, which belongs to man ought to be, buty and lowe out of with any disposition to our mind to be good or disposition.

Why then should a man cling to a longer stay her things from the ruling power of the universe and of nequently gr which thou art sipplied with men who have the same whether we examine these things, in the matter which is good me injumed. For all things are arefused about me indivity and of them, nor does heakended on the whole, and sawhat which is according to be compact, and to show no ircumon nature, so also are they more man according to nature contain something performs in it, and what value everything, changes, nor any wretch or a whore, not on thee. But in the things which are much value on each of thas which are attached about it. For they are like a sphere.

It is no, it comprehends the perception of doing what is usual and natural, desert pleasures.

Thou art a part ((rof things, and yet that any be given by the universal nature. But if it is not so. Teacher stopperithee right and moves thee or thee. It is natural that these parts and benevolence and everything of this superior to one another. That so much more than itself, that which is subject to change, and anof all the rest of the body and all things, and from thy very far as it happens to thee; the more of these things return. The poet says that which is satisfied with walking; and not to be ready to speak of the ears of corn being really good, such as prudence, temention in the heavens; and the meanspired of rente which summons him. For th which alyo if thou takest in mind, what is their nature to bear it. If, then, it happens to thee in the amazation of duty like a man worsh which still detains thee out of the universal; and when thou bast fall into the same considerations be unhappy and worse than it was before. The purpose expect Plato: But thou avolutions) administers the universe, and that again will change into oll ng like a pries who is in formed by natus according to nature contain showed in the plays which follow of things, is it he who sua disposits proper time, suffers no evil good to his deserts. Accordingly men are pained when they are asleep, or when he has run, a robber and everything on, but whether for another life. For of this disposition of the universe directs all his own attention, he very seldom stood in necessary for living and inally to from men who have the same time employ the hindrance towards, as a nimpelsoou misfortune, but that to be also a thing which falls off who hinders thee?

Observe too who the beings nothing which thou shouldst be character to rouse men's labours, marrings done besolves; and all other things have been constituto have been used, and in the way of repraising all that they give and have ordered.

Above all, ngs are in such a kind of envelopment that thou dost not succeed in doing everything according to nature. If they cannot endureasoning art (Athen and the bad. But examine, not to discover that nothing happened to thee badgent with the things which are according to the nature of the universe is a common kind (socially are of another kind from that which thou dost verthe same time that which remains all time by fixed periods (r all the vange from chairing wrong is madness, for he who repen my children; and leaves, in the case have the figtree to him who is in error is thou art not formed by nature to bear juice in the figtree to bear juice in indeed there is also one perfection for lived well. But he may beso busy about of?
Consequently in the same way ought we to act contrary to my god and ds necessity to say to any over thy mind, ith is the most life.
The idleasures is a concurrance and three from pain and uleng society harsh words.  To look for themfain a reference to thee, origination: and a man thou most common as; and believer thou hast seen Euppeasing, d of we him, surely in cons?
Too see what things are in thy man who will not follow after.

The indelably to nature, though I still fall short of it, and what knows beginning to himself thanquility.

What a thing of these men who ymall degree, out of the universe and of time to be his value. For look to this only in all cases call that a be been tar, and he did not reproachfully, nor yet about which from all other things, and fault with them:; and cheerful; and to have no leisus; for even three hours so spent are sufficient, I use it for the work as an instrument thing to thee or to which of content, lf reliberate of each still thoughteal, neither intelligent nor social, it proper to adm, the most common slaese from corrupted cours be deprived, if it is true that this is altogether intelligent soul vaingus, nor a great providence and echou art a part, and of what administratter without comprehension, thou must not want then who have the rest of men, but looking at and reflect into such a bractire, which is not a decommon in the heaving, so also ought we to propose to ourselves at things, or is, to be permitt according to nature. In contemplation which stul re common interest, nor without any range with the affects and hypocrisy and luxury as t not dy; and I shall be near to thy departure, ng to have, and what Succession of the eveniarion for theings from the universal; ander and men who fromscriting sup; and when thou hast looked on thyself, think of the whole, and safirst grew together with the tree, and has continued himself its do. But if thou dost not succeeds hich it does not can very much according to its value. Remember this too is one of the things are three of which thou art composed by consumed by cordicion and infinitely that some men and vexation and when it is possible for thee to understand amonger have think of ourself to be such as thou want when thou hast detected them, thou shouldst wippolutions change of opinion must proceed only from a certain reference to circumstances, that they do soon thing on a mounner with the things which happen daily charged for thee from all eternity; and the implication of dying is dissatisfied because something has been compounded of the whole; for no harm. And let nothing else, not even for a hun according to nature contain something passes fou want which everything that happens in the world. And the parts which are held together But if thou wilt choose to understage of the voice useful his becomidy, and it comes from some something like what the gardenerest to it. What does the earth remembering, one thing and diffusion is the reason of our constitution; for a man then would those do after thee, and the longest which a thing of this

# 150 SEQ_LEN
## 64 Iters
### GRU

From Playost thou be nourished, unless the deaves in contain things which it is carried; and fearship of reason in all intelligence puts itself in motion once, and everything else is of the same kind as that. For in thyself also, that which lasts a quiet it away freedor at the same time, thou wouldst be tranquility.

Wipt is contented with thee.
Remember how many things who have seen thee has been compounded, and into which he and come and fee; and by what Plators of the universe a vine. Whatever happens to every man loves himself mode them like this: If they were towards the gods, nor tirm men they are quite ripe, gape opentutut if the eyes not the man who with the aid of mind: and to be stronged by any pleasure, and in a manner thou doest, do notice, who pervades all things, and one and nature. But if it is only universe, since thou art a bad man do st; and to bried into what a fee in ride, hus like a mariner, who has doubled thou art dying and on the poor flesh which has grown about thee; for the pasts of sequence is an evil to the anization of the same kind. And what its cault with the airy to want of Comprehension to joy inout, and how many heroes after kindin superflook remember that the remembrance of thee: Neither broo hastice lead the many to be good, he will listen and readily receive as very applicable that which appears to give thee pain, thou thyself standest in perfect the things which are not well; and aer all things he knows betinning to the desire of man is part of a nature which has not perception or reason, and is subject to be disple with a generals after them, and to hate them immortal? Was it not in the order of destiny that the whole court of Phispisus, he is there be the same.

Think of the attained by shameless called for mankind; he considered himself no marm to be unhappens, and a man should hold on to the ofitter there is of the same kind. And what I am indebted to him for being acquainted with the disposition of the universe, and in the wen's offension; and let neither another man's words, and thou hast forgotten this, that everything whichly arvost is ching. And are the chooseso, for it is making a man has been spectandly divine, and fame; and one thing he of men and the conformable to reason.

Be thou erects refly this, that I belong sensations, to the soul a womphose after all, which we have no power or they have possible; and Let not at life. He was also easy in convers away from the feed of e ght ast, nor about the word, or by stronge, and above all do not conferred the parts or there is opinion.

If it is according to nature.

This I never into another, why should a man have any obstacle of the kind absolutely requires a disciplines of my neighbour is jast or understanding, and thy power not to think so.

Whatever any one dost not unsounce and never intelligence and the stare, doing nothing, for it is the peculiar quality of anything contrary to the reason of our common nature; and I quit it. Why dost thou thou hast reany a reasonable anion, ite it has given to each man not good; and that no done many other things opponent themselves in abject way what is not in thy power to live no longer, he can live present, then, for a nature which has been made should be done by one another, and litter and scolding from their elevantagoop which a man can be better than such thoughts as these: for it is the peculiar office of the rational and intelligent motion claimso useful to me. Whatever man has fixed bounds to this too: and all ought to have the intelligence which a life than a certain persuasion, as of Dimity to resist the need of another it is past only on that which thou doest: and remember...

A cased cause of everything is so farofration: for what a man has not, how can any one ismed atoms, or ansign of its ridd.

When a man kind of men they are at bad him his eyes; so that if a man should live longer, it is fall of nature, he is a child. This, however, is not only an operation of nature, but it is also a thing which leads thee that every man lives the present time only, and how many pleasures and pains thou hast seen: and how many court of Histomething, and in him I observed not have their fame, and how many dysind how he may be well to be atraid nature for things to each movement and apterned by nature of an extension: And further, let the dancing art, or being man: there is best with the air which is useful and well suited to society more my life under thoublection is a common nature through they about which it was on the plan's shall exist in thy ruling part and died. There is was propried to hold out against even thoughts as these few danch, and direct the bodythings which take place now: accepting, but with the common nature judgly about thee, another rules made rational animals (concord with my hit which is say of equal in the world. And how religious be in the world are already gone out of it? And of this and ther for my children ifflet thy fearing and ignorant mind; and not to be regulate and comely for thee usten action and for my bringing upword at least, and the time is a pestilence of animals on the universe, so far as it has wander away from him whose doil is a man after them; and thy labour which all things is the constitution of plants. So then that which is assigned to thee distract they good arm is Hear to us through the less, and to its object.

Either the last a disposition to do getrater's of the universe and I say there we may be reminded or an opinion or not. But that a man has assigned to him as his child and help to all, so ke praise thee. In a wondrous Parn of such eatht keep his movements of their continuasterWion of things, and anoughted without affectation, and to long with him who rightes himself belong sensations, to the soul, nor can they thou mayest be completely extinguished?

What hinders the mind indeed, bonesimity by a sumple mort worth value.

See that thou seest consider thus: In the herd of what parts happen to thee by themselves, neither suffer in thy death will not seem to him to be a governmembered: for thou wilt consider how many the holsestion of necessity be unite and compose itself and of mohe survied im the compliant; and that Magnanimity is the elevation of the intelligent power is no evil and objects, who is in futy with which there is the same lad do not be in holoor and greatled out their bearing about a corps, his donations to the people, and in seling his perstances, that thou mayest live in happen) is a necessary sequence, not refuse them togh things, in the adise than he was introduce, the healing of him who does wrong, that it is illumination. For thus thou wilt use them even held they have happened or back agrieverate, and that I did not make proof of my virility. For thus thou wilt have made the intelligence principles only: the one, that nothing which is no activity, all these proper to a whole briefest precept is sufficient, and any common precept, to remind him that he natusely dyposith keep mishounds, Leink of the ghodyed, and it will ever bubble up, if thou only circumscribest it, and changes are compounded. But if this is so, consider thush I am Anding, but is sufficient, yeeping to thee very much with things, and in change far that of the thoughts, and of that which is incible no violent to thyself, my soul; but thou wilt no longer intendionally, nor even separated has done true.

He he who does it through the sensations of the elements out of which everything has been orsianedf character.

From myrengs he mush by means of rational men. Sound of men's violent, nor, as one may say, from the food and the air good to that which is advontion of the Formal For a showe thee, another prays: How shall I not desire to be released? Another that for this reason that it has its proper tare also to some thing goes or move the healthy understanding ought to be peried of any other nature; and a sumeloiverse it is diserse then.
Let the common nature brings nothing without suppose that I was subjected to have goncorride, all this is fixed and deeper insight with usest thyself: How than are not do that; for those parts which have felt will complain, if they choose on eternity.

Accordingly then the ruling principle? And its end is than that is, sincently the breath all thy very activily, and not at all for a thed, and is earth and bedidence is hander? The bid thou mayest have had thou shouldst not rather blame thyself, because he who trans thee art; and be quiet at last. But perhaps co change and consists in yeets and objects, do thou, since thou hast praceded in an activity which is the constitution of things, and indeed they are embered all that happens, as necessary, as usual, as flowing from a principle and soricient and course we value the clapping of tongues, firth, the hindrance towards the ancirceive according to nature, and what kind of a life that is, and the differences which the moso is the death of a wilt good, since it is within, and, in jest and act considers another man's act there no virtue what is the same dong worthy of blame, nothing which is in conformity to the nature of the universe and their crant and takes back all, to nature, they must then I have had my reward. Let this always be presing them his skill in pervicient reason. For if he should cast come from without a display; and to y all these things for him that he was harsh to none of unness by virtue and the adraid of any other nature that each thing is moved in the thread which is subjected and presented to thee, the purpose, and the time way in thy death of every separated and cursistent those of those things on the operations of nature, clinging to something else...

Let the universe is extend is so constant ch form, and is lead things be quiet at last. But perhaps he has not done wrong.

Either all things proceeding or bodies of those who have the faculty of comprehension or understood it.

Asecher is many things wrong, and care for what they do wrong it to be something

# LSTM SEQ_LEN=250 EPOCHS=128

From bus not been remedle for everything insincerely and untruly existung and other things, nolly and be made worse; but that which is evil to thee and harmful hes to show it hazave his life inconsiderate, opinions, it will see diverted from thy course of action and to give way through fear; for both are equally deserters from their post, the man who does it through fear, and the many to be made which is not formed by nature towards all things which it is wild.

Ret thyself in the common interest, he has acted conformably to his constitution, and he gets what is his own.

Wilt thou, then, if thou woolived masters or from him solachifes is intelligence with the common notions of man's naturears of life. Let him live is not complete and thy wronged man, but we may say, I would not even and hust with e respect to the gods, nor deligher in a point, and the universal nature has made rational animals for the sake of one another to help one another according to their deserts, but in no way to injure one another, he who transcitation.

How then shalt thou not see then that further thou wishest to receive freedom from profitable corren to another, and the exhibiting with the rational animal. And this faculty promises freedom from hasty judgement, and friendship towards men, and obedience the man who has antry of his unially, when they contrary the letter which has been made showed by pleasure and pain and the common nature; and the causes which are useful to the parts at the parts which can feer the obstalle court, why has attended ther to thy order hat God may give him.
With respect to that which happens conformably to nature, we ought to blame neither gods, for they do nothing writh the universal nature, and despise the bleat is (the principles) of their ortale or a Andation and soul is dissatisfied with thy present lot, or shrink from the future.

All that is from the gods is full of Providence. That which is from fortune is not separated from nature or without an itself naturally exists in a man would not be recong to the understanding of things from the many which is presented to thee in life, and always to look at things so as to see several and comes in the sure of the acts of life, the mind man should art by repectation the ruling principle is that which rouses and turns itself, and while it makes itself such as it is and such as it wills.

In conformity to the nature of the universe every single thing is accomplished, for certainly it is not in conformity to any other nature that each thing is according to the nature of a reveralle, there no harman satisfied with that which that which parts a partion is in the morn nature requires. He this is not reason of any other poor soul and reward of what shall nature is no man who belowed right fast value changes, and what kind of a thing it will be when im his complain? But about a man has the power of acting conditionally and changing from distrance. For what more wilt thou know what such good.

How then substance and one and the Cassu, such good.

What is thy art? To be good. And how is this accomplished well except by general vire except except exery to do my writersind y the praise, which has not been instructed and pride. And also certain this art, like hand, and when he has takned the badk (of a right line, and as it were is divided with the things which are undiviuged to anything else, but it is common polinich died and then Coeserfal appleas in each acts. Sound. Why then do you not goor for the best faculty now to me? And of what nature am I now making it? And for what purpose am I now us? They are quite to endure in every act which thou doest: and remember.. Look within the whole remerral carrount, and show him his error, ad of the universe is obedien to the universal nature. But if it is not sufficient, then either I retire from the work and give way to him who is able to do it constitution.

It is natural that these things should be done by such persons, it is a my lothout in it, and end; but what is it to the riving faculty now to me? And of what nature ammonatus and men as theye are consistion winh the reason will be carried through all things, as fire upwards, as a stone downwards, as a cylinder down an inclination to philosophy, I did not fall into the jade which knows not the things do not prociee, and does the work and give way to him who is able belongly relighal intelligence with the composition of the body, as not to have allowed thee the power of circumscribing themselves; and my men whole right fashed or not received and curtames as if set something to killed; but the other always has his hand, and needs to do ast in comparing with city and cooperators in the things done a fortw, and too what through the about nother s one unitothem, so as than which has strange the tide.
From the better the parts together of another.

Thou hast not leisure or baingly its rysolver; by the acts that which thou hast often seen. Every wanth of possesses the sparns, and by the power of readines all things so far as it just. For such a thing is he sich.

Witn a social animal, so far as I an ig, being himself yeans or the truth vigerally even the most and justice, and the graniato bleashing were help, and every man who is going to do them by readiness; to be red abtiful them; and what reason of sure of those bodies which continually do the same things and in the same manner perform fame.

For one man and the good and justice within the same. O content with thems to a disposition to do good, and to hindelle mind with indince they shall nature has observe how changes and consistent as well asong, if any one in pieces many truals power wrething, which has produced thee, and thou wilt cease to be a stranger in thy native law, and again makes other new things from these very same, so that she rid of a fight and of a name. But if thou dost not this consists in keeping the daemon within a man free from violence and unharmed, superior to pains and pleasure, and besides these, as Epictetus, he facus die.
All things are such a thing, as anything best into content with everything that opision man respect to pleasure and pain and the confusion, and as herse this understanding to its. Let this be the state of thy affects, and thou standest erect. To recover thy life use of the each fame, and even this only continuing to exast and acts wather or which are disagreeable to us are the same as pain, such as excessive drowsiness, does its own appearance, and end of such nature (or soon as thou wast notly rese disposition in the gods, and what serves it is both independent of the will and not opposed to the general interest, but it is good, since it is shald the acts which reason or a disfortune than thou wilt live the court nor carried up.

Penetrate inwards into men's leading principles, and thou wilt see what judges thou art afraid of, and what kind of judges they are of themselves.
All things are changing: and thou thyself art in continuous mutation and in a manner in continuous sure or a tears ersions, those which concern the power of forming an opinion about these things, will suffer nothing, for it will never deviate into such a judgement. The leading principle then in the matter of the anishander and Guttle grape age.

How our in thy opinions with given thyself absto How that which happens to thee to the truth; but if thou canst not do even this, of what use is it to thee to find fault? For nothing sepraishfor me who must often as thou art formed by nature to bear it, or as thou art not formed by nature to bear it. If, then, it happens to thee in such way as thou art formed by nature to bear it. But if it happens in the world, so as to see dy will remember not to because they are must. And then there is a continuous serisulable and yoe no acting to its own and to the world, any produces of her art, and that which is harmony or a sincle deads or sown grows erect, not kindle dispersion of thy endersto do nothing else.

Everything which happens either happens in such we are easily deceived and carelus and Pat themselves, at which of those who well die indeed so far as it is nor wors, to the rememaning and judgement without a ruling pact clago, unless the best place to each perchince, then be the praise which comes from doing well, but as in the mind which made requires and if thou wilt go oth come to bely and incluctial and ncignd, and what s offents its skilned, and the reason which have reason.

Let there be present to thy mind and venery and fire; nor yet in any turn or art of all thougatus, and only a soul continue or Howner than in the way which is proposed to ourselves. Sumple and worth pleasures, doing nothing without purpose, nor yet falsely and with heropos. But if a thing also is true. But what is of the nature of abuntaings which comes se do what is it.
Something experiand, and all the rest, not many things wrong as been compounded of the elements for which adore well dieply with respect to that which is opinions his either of necessity fall upen end.

Profed to change in a fiver gand. And father, I ungred to give way thou called marm of his own opinions his een of sensation one another, and the tring to the least for the than Court life, and both everything which has taken placu after dueny only which his lind ncclonger social reason; he is blind, who shuts the eyh of all things which he has distinguity, and how eman it must they to carry them thou. And all now thou hast fallen into doing as he showed in any way to the commodity of life, and when it is in his power to go, harm, so besides these things were good. But now the more of these things a man can live, thene things who has been first learned to obey rules thyself. Much more is this so in life.

Like this persation only things which are citceftle providence of any of the things which are conformable to rismusion, which had on ever saying the truth in this man bo it. But if altoge are hersain vigur and hounds; and how he likes at the sam

# GRU SEQ_LEN=250 128 EPOCHS

From they have a very vexture useful to thee as a rational being, keep to it; but if it is not sufficient, then either I retire from the work and give way to him who is able to do it better, unter intelligence with the composition of the body, as not to have allowed thee the power of circumscribing thyself and of bringing under subjection to thyself all that is thy own.
Repentance is a kind of selerg, that it should stand ready and firm to meet onsets which are sudden and unexpected.
Small matter thus little compound, thyself, must either be dissolved, or were is not a woment. For they we governs kinden and Herderent to things in those who are suspected of bodies to be superior to love from himself agart one that he who has done this act, does he suffer any evil for this reason that the act has ceased. In like manner then the whole which consists of all the acts, which is our life, if it cease at its proper time, suffers no evil for this reason that the act is the velunable hands of me what is agreeable to thy own opinion to move for the respect and the same activity to be relatedixthone, or the deaths of others; and how many things callen moves.
Then thou wilt be a man both everything which has tell of them, show him his error, admonish him. For if he listens, thou wilt cure him, and there is no need of anger. Neither tranquility saying with it.
"The earth loves the shower"; and "the solemn aether loves": and to he wild beasts, as for thee and prescribed for thee, and in a manner had reference to thee, originally from the most ancient causes sw? What is there now in my mind? Is it fear, or suspicion, or desire, or anything of this kind.
Nothing happens to any man which shearninus ryady are now to be separated at all from the saying of the superior; for the universal nature is the nature of things, thou hast not yet been injured nor instructed and moves to maintain this nature. Constantly and, if he has such armpits: it is necessary that such an emanation makes us neither brood or behavour naturally exists in according to its will, everything in thee is either differences another dies.
If generate ignorant persons, and those who form opinions without consideration: he had the power of repaticity to be removed and pained as an end of life: to which a man ought to come pure, tranquil, ready to depart, as best to thee to with the Common intelligent nature gives an abundance and in the heavens; for all things are formed by nature to change to be produced"?
Thou art satisfied with the amou mayest know whether he has acted ignorantly or without an interneleable to lay does not flighted in things indifferent things: it is in my power not to become good tomorrow than to be good or bad, there remains for it is union: this changes of his bedo insure, and that there is no obedient to the a continuous spinning of the thread and the contexture of the web.
Thou art a little soul bearing come down, nor it does not allow, or in the way which it does not allow, or what ill of thy life by a man who cursest higher to imagine a deity without forethoughts; and about to behave I have good or evil, thou wilt more readious to pleasing to each of these things; s it cannot be over of the things which thou never the soul incl mannique to its compelled to observe what is doing. And in the one thou shouldst see immediately to what endure die, and then Secunda died. Something like this leck immediately or remotely to a social end, this tears asunder, and then for this reason renounce himself carried up.
Penetrate inwards into merf about this alone, and so away, is a reason to amaller that these persons too are external and remain immovable; but our perturbations come only from the opinion which is also a very vexation wators of the reckonance which is sets only those things which are conformable to its nature: still it does not submit, but is carried in the opposite direction. For the movement towards injustice and intemperance and to anger and grief and fearthy enter of philosophy, but to be hindered by another; and to hold good emotion.
They we many a universe desire.
Seckndainest to its constitution; and to slake of the things which are asconstitution of man?
At first tragedies were brought on the stage as means of reason are so many god and the worship of its excellence, acts no tragic part, does not groan, will not need either solitude or mutual involution of the who assumed it pursuits, and quasill be endired by Phal; and that the hissolution and not to be vexed at the dear according to the nature of the universal; and in bark now is a proper work, and as thou findest many other differences at last that for this reason.
Let it make no difference to thee whether thou art cold or warm, if thou art doing thy duty; and whether thou art drowsy or satisfied with sleep; and whether illspoken of orter; but rather thou shalt be reconciled to recover the kind.
Nithout d, and he who loves fame, and he is perturbations also in the universe; and this is the present. But this is reduced to adeve them. And his affectionate; and how is altogether one another in some kind of connexion. For there is altogether one fitness, harmony. But wilt thou be angralent part of we are made for this common pleasure, and that I priso indeed consider the same universe devoid of gods or devoid of Pood from thyself.
Every matters a both thou, hinds, these things then to those who can content with the things which are ordered by Providence. From thence all things flow; and there is besides necessity, and that which is for the advantago of the same kind as reason is an evid that thou art a man like o clears know where he is. And even those who are completely ignorant understand what I mean, for they say, It (necessity, destiny) brought this to such a person. This then was brought and this was precribed to him. Let us then receive these things, as we may say, write them in ourselves, it being in our power, if perchance these judgements having hindered themselves, nay even sound and one such good.
What is there new; and that I do it as we are sake of the substance, and to chorse to one end? which has from det the bllosither saying anything containest the opportunity of honouring thyself. Every man's life is sufficient. But thine is nearly extends itself into the infinity of time, and embraces and comprehends the period of each of the same stock and participate in the same reason.
Everything material soon disappears in the substance of the whole; and not then imagine that they are of another kind from that which thou dost venerate, but form a just porte, if a man should examined alappens. And inasmuch as I am in a manner intimately related to the parts which are of them has hade men, since they he haviI no. but then do the universal nature has made rational animals for the sake of one another to help of the divinity. But if there is a child, said Epictetus, he should whisper to himself, "Tomore of the things which are assigned to me out of the whole; or there are that child. What a thing from such a material! Again, the child shall come out of thy wife's womb, so be reason either political, and, as thesee. What is there now in my opinions if they ching, and besides this, they also take care of them in all ways. But the fourth is when thou shalt reproach thyself for anything, for this is an evined apon or such and surery, things then, and that his friends rudor through a narrow opening into a drain of a philosopher; and thy plan of liotimus is manly and aghe intendiving away from the observation of our own ruling power. We ought then together is of the same kind as thy health. And should not rather brought on the stage as means of reason and diffused with anything that happens, this te, a flux by man incride, nor inor the discourses of Mendlaction that may before thee.
How cruel it is not to allow men to strive after the things which appear to them to be suitable to its nature, and in short of a universal cause is like a winter torrent: it carries everything along with it. But how worthless are all these poor people which things pass by one who is really a must, nor yet to water nor to from other life. For thus thou wilt not fail to obey reason, and thou wilt repose in it. And remember that thy death also in the whole body. But all of these things s be unatus and come to which everything is opinion.
I and have those before us seen anything more, but in a manner he use of men; the other, to change thy opinion, if there is any one at hand who sets thee right and many other things of which thou canst not say, I wish for green things; for this is the reason why such acts are names are plant and takes a view of all time and of all substance, dost thou suppose it possible for him to think that human life is any truth: if thou non either of the matter, who is the same to those who cause the very extinction or contant; what is therefone in my opinions, And firthy itself alone, and whatever judgements to the life of the Universe loves nothing so much as to change the things which are part of been consideration, keep their fitseffor solforithee are speaker he deed by the other hand now that he says, who find fault with what happens and those who try to oppose it entirely and unjust come for what things; that it, why should they have any desire towards that? For what advantage would result to them from this or to the whole, which is the special object of their providence? But if they have not desire and indeed course of the whole; and everything formal (causal) is very soon to be displeased; but it is a runaway. And he also who will repurt for itself also to who is efforted by him who rules all things, and he is Law, and assices a romber by desires.
Thy aerial part and all the fiery parts which are mines acts or benetoccersal and post.
Empty you see that these thirgs which are useful to thee as a rational being, keep to it; but if it is only useful to thee as an animal, say so, and mainta

# Final
<ul>
<li>GRU</li>
<li>SEQ_LEN = 200</li>
<li>EPOCHS = 128</li>
<li>dropout = 0.1</li>
</ul>

From the formative principles (forms) of things above, below, and ago.

To the aids which have been mentioned let this one still be added: Make for thyself a definition or description of the things which take pleasure in one thing and rese thing because it deprives thee of this.
When another blames thee or hates thee, or when men's occasion to repent and a man to depart, hoping perchance to get so mingled the intelligence with the composition of the body, and return able to do it better, unless they will be conformable to nature, so also are they mo; and to have toe into existence out of nonexistence. Every part of me then will be received back into its seminal principles of the universe, so far as it can. For to be vexed at anything which happens is a separation and unimpeded, if it does not disturb and impede itself.

Eudaemonia (happiness) is a good daemon, or a good thing. What then dost thou think of him who ave in harmony with what is really good. But if a man has first conceived as good the things which appear to the many to be good, be judged by thee to be of the same minf all the external circumfluent vortex whirls round, so that the intellectual power exempt from the things of fate can live pure and free by itself.
I do it and accept its own activity, and yet have no opinion about a thing, and not to be d to definite periods of revolution.

Reason and the reasoning art not chaised and wnot soon disappears before thy eyes those to whom the same things happened, and how they were vexed, and treated them as strange things, and found fault be applied to him which is proposed to them; and then, if being lame thou canst, correct that which is the cause of form, the formal.

Do not be whirled arounding from their elevated place. But a short time ago to how many things cause firm and remains unharmed. It is a mant disposed towards them, for by nature they are friex. How then ought to be required also in the whole body subject to putre after themselves. Altogether the interval is smell with Good on a double man Has reason, it will be said, and he is able, if he takes pain, to discover wherein he offends I wish thee well of thy discovery. Well then, and thou hatter then who have perception and motion, and in object, if they can cooperate f the figtree produces figs, so it is to be such as thou hast hitherto behaved towards their bries, and to its object.

Enter into every man's understanding and knowledge?
That which happens to (or, suits) every man is fixed in a manner for him suitably to his destiny. For this is what we mean when we say that things are suitable to us, as the workmen say of squary like a person who has been first wronged and through all bodies towards this, even if it brings nothing which may not be borne bedings what is now said: and for this reason go away less kindly disposed to them, but usual and apt matter to work on.

The perfection of moral character consists innt be afraid to lose him? In fine, turn thy prayers this way, and see what comes.

Epicurus says, In my sickness my conversation was not abott.

How also ir the things which are in this, to change them, to take them away hence, and to carry them there. All things are both familiar and what thou art a part; and that of thy neighbour, that thou mayest know whether he has acted ignorantly or with knowledge, and the things will proper to himself, understhooug thyself not being powers with due consideration, keeping to that which appears to be his own good; and he presented the appearance of a man who could not be diverted from right rather than of a man who had been improved. I observed, too, that no man could ever think of those who live with us and present themselves in abundance, as far as is possible. Wherefore we must keep them before us.
Thou art not dissolution and to be united and to resume his place as a part.

As the nature of the universal has this work to do, to remove to that place the things which are in this, to change them, to take them away hence, and to carry them there. All things are made for cooperation, like feet, like hands, like eyelids, like the crueation of a philosopher; and thy plan of life also opposes it. If then thou hast truly seen to hinder in those who are produced. For they will certo been, and they will be in a manner, they move our pity by reason of my learnings. But if we juggle who is chastened and purified thou art a Ran our word, thy blaze these agitations with respect to the things which are part is a runaway. And he also who is grieved or angry or afraid, is dissatisfied because something has been or is or sensual enjoyments at all, nor to be condect to all for the fruits of the whole, everything that they do not live according to net there be soficher it goes happily on its road.

How strangely men act. They will not praise those who are living at them to thee indeed have not been stupid nor deformed in body; that I did not make more proficiency in rhed truth, fid thyself in conformity to thy nature and free from passion, and also consider that his ruling faculty is a man's, words of a kind disposition with the universal: and in a little time thou wilt be nobody and nowhere, like Hadrian and A man's wickedness hinder thee, nor opinion nor voice, nor yet the sensations of the plant; except that in the plant the nature of the leaf is part of a nature which has not perception or reason, and all hypocrisy, and selflove, and discontent with the time is a point, and the shorn with him whore. How said by any of these things should not happen or happen? for certainly disposed towards all; nor doss thou in this can be mothe become fame, and even this only continued by a succession of poor human beings, who will vere neither substance from without nor wants a place into which she may cast that which decays. She is content then with her own space, and her own matters which are indifferent, no less than the sun or wind or a wiodence or showing like, not because himself. In the next place, the soul does the and material and not a politically moved towards things because they suppose them to be suitable to their nature and profitable! And yet it is of our free will.  Epictetus also said Epictetus, he should whisper, the life now lived among barbed by them. He was also easy in conversation, and he made himself agreeable without any offentian belongs to the Grecian discipline.
From Rusticus I received the impression that my character required improved my owh philosophers, not a flowenor putrit has its reason too. And again
We must not chale and further thou hast forgotten this, that everything which happens, always happened so and will happen so, and now happens so everywhere; forgotten this too, how close is the kinship between a man and thing.

One thing only troubles me, lest hour comes, that he has either humbled by them or letting them pass unnoticed.

From Sextus, a benevolent disposition, and the example of a family governed in any thy hour; and that no power has ever been found in so many gods too is moved towards things be quiet, through piou soul coverings; the result is that all comeliness dies away, and at last is so completely extinguishedueshatten? Enothing why do change, and they will either be reduced to vapour, if indeed all substance is one, or they will be quiet at last. And ingeed these things must be accomplished thus, and that even they bear them who cry out "O Cithaeron." And, indeed, some things are said well by the drady and finishing the thread of thy being, and both all employs equally nature? What is there now in my mind? Is it fear, or suspicion, or desire, or anything of the kind?

For indeed thou art a man like others; but that me not discourse on the nature of things as before, keeping to this main point, how the mind, while participating in such movements as go dois maild, and wilt make thyself like Empedocles' sphere,

Al

composed to the animals which have no reason and generally all things and observing the rapid charpe of things, or remembered itself nor complaining.
Another may be objected, Why what is more agreeable than this which I am doing? But is not this thought be present, that there is no dishonour in it, nor does it make the governing intelligence worse, for it does not damage first.

We ought to be, but be such.

Constantly contemplate the whole morals of those who live with us and present themselves in abundance, as far as is possible. Wherefore we must keep them before us.

Thou art in continuous mutation and no more use it falls off when it is ripe, blessing nature who produced it, and thanking the tree on which it grew.
Be like t me the world produces such and their sports, and poor spirits carrying about dead bodies, such is everything; and so what it changes, and without reason, but to try to restore either inconsiderately or otherwise than as just and underity and modesty, and to be done without a purpose.

That which has grown from the eager of thy life in thoughts then to that gleasure is nor considered his experience and his skill in expounding philosophicity, and my not external help nor the tranquility which others give. A man then must stand erect, not be kept erect by others.

If thou findest in human life anything better than thy own mind's selfs the deity which is planted in thee, and disorder in the body, he cares not at all: for even if he must depart immediately, he will go as readily as if he were going to do according to his own nature. And it is in his power even who shy perchive it has in the universe, and how long it is formed to least on account of men's involution of things, and a dispersion; or it is unity and order any work on another suphisie might penedrath the soul of mankind. Does anything happen to me? I recoment is deity and by what thou hast not as of what thou hast: but of the things which thou fearest of all the overcurious feeling and the movements (desires), and in a word wrongdoers how strange I true philosopher, that the offences which are 
